Hello... In this notebook I am trying to show whole pipeline from episodes scrapper to applying machine learning models...Honestly I didnt know how to do this.. even i last competitions.. I just saw them as crap.. But in this competition I decided to learn it and learn how we can apply ML models on scrapped episodes data:/
> So first giving out credits to those notebooks I was able to learn and apply.. Thanks to them a lot.. Please check them as well:)
> > [Felipe's notebook](https://www.kaggle.com/felipefonte99/ensemble-pull-vegas-decision-tree-model) <br>
> > [Robga's notebook](https://www.kaggle.com/robga/google-football-episode-scraper) <br>
> > [Lebroschar's notebook](https://www.kaggle.com/lebroschar/generate-training-data)

# Scrapping out Episodes

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import requests
import json
import datetime
import time

from numpy import linalg as LA

import matplotlib.pyplot as plt

import json
import glob
import seaborn as sns

from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GroupKFold

from tqdm import tqdm

import math

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Extracting top-15 Teams episodes

In [ ]:
BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"
# inital team list

r = requests.post(list_url, json = {"submissionId":  18923141}) # 1st Rank Team
rj = r.json()

teams_df = pd.DataFrame(rj['result']['teams'])
teams_df.sort_values('publicLeaderboardRank', inplace = True)
teams_df.head(5)

In [ ]:
os.mkdir("./json-folder/")
def saveEpisode(sub_id, epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('./json-folder/{}_{}.json'.format(sub_id, epid), 'w') as f:
        f.write(re.json()['result']['replay'])

    # save episode info
    with open('./json-folder/{}_{}_info.json'.format(sub_id, epid), 'w') as f:
        json.dump([r for r in rj['result']['episodes'] if r['id']==epid][0], f)

In [ ]:
subs = [18923141,19052323,19145375,19120525,19123171,19179822,18913702,18778644,19117574,19191948,19176740,19117330,18983613,19191948,19174241]

for sub in subs:
    start_time = datetime.datetime.now()
    r = BUFFER;
    result = requests.post(list_url, json = {"submissionId":  int(sub)})
    team_json = result.json()
    team_df = pd.DataFrame(team_json['result']['episodes'])
    print('{} games for {}'.format(len(team_df), sub))

    for i in range(len(team_df)):
        epid = team_df.id.iloc[i]

        saveEpisode(sub, epid, team_json); r+=1;
        try:
            size = os.path.getsize('./json-folder/{}_{}.json'.format(sub, epid)) / 1e6
            print('Saved Episode #{} @ {:.1f}MB'.format(epid, size))
        except:
            print('file {}_{}.json did not seem to save'.format(sub, epid))    
        if r > (datetime.datetime.now() - start_time).seconds:
            time.sleep( r - (datetime.datetime.now() - start_time).seconds)

# Converting episodes JSON to required format

In [ ]:
replay_dirs=["./json-folder/"]
json_paths=[]
json_info_paths=[]
for replay_dir in replay_dirs:
    for path in glob.glob(replay_dir+"*"):
        if path.count("info")!=0:
            json_info_paths.append(path)
        else:
            json_paths.append(path)

print("replay num: {}".format(len(json_paths)))

In [ ]:
def log_training(result, n_machines):
    """Records training data from each machine, each agent, each round
    
    Generates a training dataset to support prediction of the current
    payout ratio for a given machine.
    
    Args:
       result ([[dict]]) - output from all rounds provided as output of 
                           env.run([agent1, agent2])
       n_machines (int) - number of machines
                           
    Returns:
       training_data (pd.DataFrame) - training data, including:
           "round_num"      : round number
           "machine_id"     : machine data applies to
           "agent_id"       : player data applies to (0 or 1)
           "n_pulls_self"   : number of pulls on this machine so far by agent_id
           "n_success_self" : number of rewards from this machine by agent_id
           "n_pulls_opp"    : number of pulls on this machine by the other player
           "payout"         : actual payout ratio for this machine
    
    """
    # Initialize machine and agent states
    machine_state = [{'n_pulls_0': 0, 'n_success_0': 0,
                      'n_pulls_1': 0, 'n_success_1': 0,
                      'payout': None}
                     for ii in range(n_machines)]
    agent_state = {'reward_0': 0, 'reward_1': 0, 'last_reward_0': 0,
                   'last_reward_1': 0}

    # Initialize training dataframe
    # - In the first round, store records for all n_machines
    # - In subsequent rounds, just store the two machines that updated
    training_data = pd.DataFrame(
            index=range(n_machines + 4 * (len(result) - 1)),
            columns=['round_num', 'machine_id', 'agent_id',
                     'n_pulls_self', 'n_success_self',
                     'n_pulls_opp', 'payout'])
    
    # Log training data from each round
    for round_num, res in enumerate(result):
        # Get current threshold values
        thresholds = res[0]['observation']['thresholds']

        # Update agent state
        for agent_ii in range(2):
            agent_state['last_reward_%i' % agent_ii] = (
                res[agent_ii]['reward']
                - agent_state['reward_%i' % agent_ii])
            agent_state['reward_%i' % agent_ii] = res[agent_ii]['reward']        

        # Update most recent machine state
        if res[0]['observation']['lastActions']:
            for agent_ii, r_obs in enumerate(res):
                action = r_obs['action']
                machine_state[action]['n_pulls_%i' % agent_ii] += 1
                machine_state[action]['n_success_%i' % agent_ii] += \
                    agent_state['last_reward_%i' % agent_ii]
                machine_state[action]['payout'] = thresholds[action]
        else:
            # Initialize machine states
            for mach_ii in range(n_machines):
                machine_state[mach_ii]['payout'] = thresholds[mach_ii]
            
        # Record training records
        # -- Each record includes:
        #       round_num, n_pulls_self, n_success_self, n_pulls_opp
        if res[0]['observation']['lastActions']:
            # Add results for most recent moves
            for agent_ii, r_obs in enumerate(res):
                action = r_obs['action']

                # Add row for agent who acted
                row_ii = n_machines + 4 * (round_num - 1) + 2 * agent_ii 
                training_data.at[row_ii, 'round_num'] = round_num
                training_data.at[row_ii, 'machine_id'] = action
                training_data.at[row_ii, 'agent_id'] = agent_ii
                training_data.at[row_ii, 'n_pulls_self'] = (
                    machine_state[action]['n_pulls_%i' % agent_ii])
                training_data.at[row_ii, 'n_success_self'] = (
                    machine_state[action]['n_success_%i' % agent_ii])
                training_data.at[row_ii, 'n_pulls_opp'] = (
                    machine_state[action]['n_pulls_%i' % (
                        (agent_ii + 1) % 2)])
                training_data.at[row_ii, 'payout'] = (
                    machine_state[action]['payout'] / 100)

                # Add row for other agent
                row_ii = n_machines + 4 * (round_num - 1) + 2 * agent_ii + 1
                other_agent = (agent_ii + 1) % 2
                training_data.at[row_ii, 'round_num'] = round_num
                training_data.at[row_ii, 'machine_id'] = action
                training_data.at[row_ii, 'agent_id'] = other_agent
                training_data.at[row_ii, 'n_pulls_self'] = (
                    machine_state[action]['n_pulls_%i' % other_agent])
                training_data.at[row_ii, 'n_success_self'] = (
                    machine_state[action]['n_success_%i' % other_agent])
                training_data.at[row_ii, 'n_pulls_opp'] = (
                    machine_state[action]['n_pulls_%i' % agent_ii])
                training_data.at[row_ii, 'payout'] = (
                    machine_state[action]['payout'] / 100)
                
        else:
            # Add initial data for all machines
            for action in range(n_machines):
                row_ii = action
                training_data.at[row_ii, 'round_num'] = round_num
                training_data.at[row_ii, 'machine_id'] = action
                training_data.at[row_ii, 'agent_id'] = -1
                training_data.at[row_ii, 'n_pulls_self'] = 0
                training_data.at[row_ii, 'n_success_self'] = 0
                training_data.at[row_ii, 'n_pulls_opp'] = 0
                training_data.at[row_ii, 'payout'] = (
                    machine_state[action]['payout'] / 100)
            
    return training_data

In [ ]:
%%time 

act_lis =[]
replay_group = []
rows_list = [] # list of dict

replays = []

training_data=[]
for replay_ind, path_replay in enumerate(tqdm(json_paths)):

    try:
        json_open = open(path_replay, 'r')
        json_load = json.load(json_open)
        result=json_load["steps"]

#     print("Data")
#     print(json_load)
    
        training_data.append(log_training(
            result, len(result[0][0]['observation']['thresholds'])))
    except:
        continue


In [ ]:
# Save training data
training_data = pd.concat(training_data, axis=0)
training_data
training_data.to_csv('/kaggle/working/training_data.csv', index=False, header=True)

shutil.rmtree("./json-folder/")  # comment it if dont want to delete json directory

# Machine Learning Model

In [ ]:
import pickle
import base64
import random
from catboost import Pool, CatBoostRegressor

FUDGE_FACTOR = 0.99
VERBOSE = False
DATA_FILE = "/kaggle/working/training_data.csv"
TRAIN_FEATS = ['round_num', 'n_pulls_self', 'n_success_self', 'n_pulls_opp']
TARGET_COL = 'payout'

def make_model():
   
    data = pd.read_csv(DATA_FILE)
#     group_kfold = GroupKFold(n_splits=5)
#     for train_index, test_index in group_kfold.split(data):
#         print(len(test_index))
    x_train=data[:10991129] #  80% in train
    x_test=data[10991129:] # 20% as test data
    train_dataset = Pool(data=x_train[TRAIN_FEATS],
                     label=x_train[TARGET_COL])

    eval_dataset = Pool(data=x_test[TRAIN_FEATS],
                        label=x_test[TARGET_COL])
    params = {
          'depth': 8,
          'learning_rate': 0.1,
          'random_seed': 42,
          'iterations': 1500,
          "use_best_model":True,
    }
    model = CatBoostRegressor(**params)
    model.fit(train_dataset, eval_set=eval_dataset)    
    return model

# Train model and save as a sav file.
model = make_model()
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model

In [ ]:
data=pd.read_csv(DATA_FILE)
data
y_pred = model.predict(data[TRAIN_FEATS])

In [ ]:
len(y_pred)

In [ ]:
len(data["payout"])

In [ ]:
from sklearn.metrics import fbeta_score, accuracy_score,roc_auc_score,log_loss
# accuracy = sum(y_test == y_pred_max) / len(y_test)
print("Accuracy : {}",roc_auc_score(data["payout"].round(),y_pred))

# from sklearn.metrics import fbeta_score
# print("F1 : {}".format(fbeta_score(y_test, y_pred_max, average='macro', beta=0.5)))

In [ ]:
feature_importance = model.get_feature_importance()

feature_importance

In [ ]:
from sklearn import metrics
res = metrics.classification_report(data["payout"].round(), y_pred.round(), digits=3,output_dict=True)
df = pd.DataFrame(res).transpose()
df=df.drop(['accuracy', 'macro avg', 'weighted avg'])
df.sort_values("f1-score")

In [ ]:
%%writefile main.py


import pickle
import base64
import random
import numpy as np
import pandas as pd
import sklearn.tree as skt
import sys
import os

import random, os, datetime, math
from collections import defaultdict

# Below is needed to submit tar.gz file to Kaggle.
sys.path.append("/kaggle_simulations/agent")
working_dir = "/kaggle_simulations/agent"
path_to_model = os.path.join(working_dir,"model.sav")

# Parameters
FUDGE_FACTOR = 0.99
VERBOSE = False
DATA_FILE = '/kaggle/working/training_data.csv'
TRAIN_FEATS = ['round_num', 'n_pulls_self', 'n_success_self', 'n_pulls_opp']
TARGET_COL = 'payout'
filename = 'model.sav'


class GreedyStrategy:
    """Implements strategy to maximize expected value

    - Tracks estimated likelihood of payout ratio for each machine
    - Tracks number of pulls on each machine
    - Chooses machine based on maximum expected value
    
    
    """
    def __init__(self, name, agent_num, n_machines):
        
        # Record inputs
        self.name = name
        self.agent_num = agent_num
        self.n_machines = n_machines
        
        # Initialize distributions for all machines
        self.n_pulls_self = np.array([0 for _ in range(n_machines)])
        self.n_success_self = np.array([0. for _ in range(n_machines)])
        self.n_pulls_opp = np.array([0 for _ in range(n_machines)])

        # Track other players moves
        self.opp_moves = []
        
        # Track winnings
        self.last_reward_count = 0

        # Load model from other file
        self.model = pickle.load(open(path_to_model, 'rb'))
        
        # Predict expected reward
        features = np.zeros((self.n_machines, 4))
        features[:, 0] = len(self.opp_moves)
        features[:, 1] = self.n_pulls_self
        features[:, 2] = self.n_success_self
        features[:, 3] = self.n_pulls_opp
        self.predicts = self.model.predict(features,prediction_type="Probability")
        

    def __call__(self):
        """Choose machine based on maximum expected payout

        Returns:
           <result> (int):  index of machine to pull
        
        """
        # Otherwise, use best available
        est_return = self.predicts
        max_return = np.max(est_return)
        result = np.random.choice(np.where(
            est_return >= FUDGE_FACTOR * max_return)[0])
        
        if VERBOSE:
            print('  - Chose machine %i with expected return of %3.2f' % (
                int(result), est_return[result]))

        return int(result)
    
        
    def updateDist(self, curr_total_reward, last_m_indices):
        """Updates estimated distribution of payouts"""
        # Compute last reward
        last_reward = curr_total_reward - self.last_reward_count
        self.last_reward_count = curr_total_reward
        if VERBOSE:
            print('Last reward: %i' % last_reward)

        if len(last_m_indices) == 2:
            # Update number of pulls for both machines
            m_index = last_m_indices[self.agent_num]
            opp_index = last_m_indices[(self.agent_num + 1) % 2]
            self.n_pulls_self[m_index] += 1
            self.n_pulls_opp[opp_index] += 1

            # Update number of successes
            self.n_success_self[m_index] += last_reward
            
            # Update opponent activity
            self.opp_moves.append(opp_index)

            # Update predictions for chosen machines
            self.predicts[[opp_index, m_index]] = self.model.predict([
                [
                    len(self.opp_moves),
                    self.n_pulls_self[opp_index],
                    self.n_success_self[opp_index],
                    self.n_pulls_opp[opp_index]
                ],
                [
                    len(self.opp_moves),
                    self.n_pulls_self[m_index],
                    self.n_success_self[m_index],
                    self.n_pulls_opp[m_index]
                ]])
            

total_reward = 0
bandit_dict = {}
my_action_list = []
op_action_list = []
op_continue_cnt_dict = defaultdict(int)

def agent(observation, configuration):
    global total_reward, bandit_dict, curr_agent
    
    if observation.step == 0:
        # Initialize agent
        curr_agent = GreedyStrategy('Mr. Agent %i' % observation['agentIndex'],
            observation['agentIndex'],
            configuration['banditCount'])
    
    # Update payout ratio distribution with:
    curr_agent.updateDist(observation['reward'], observation['lastActions'])
    
    #pull vegas
    my_pull = random.randrange(configuration['banditCount'])
    if observation['step'] == 0:
        total_reward = 0
        bandit_dict = {}
        for i in range(configuration['banditCount']):
            bandit_dict[i] = {'win': 1, 'loss': 0, 'opp': 0, 'my_continue': 0, 'op_continue': 0}
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        if last_reward > 0:
            bandit_dict[my_last_action]['win'] += 1
        else:
            bandit_dict[my_last_action]['loss'] += 1
        bandit_dict[op_last_action]['opp'] += 1
        
        if observation['step'] >= 3:
            if my_action_list[-1] == my_action_list[-2]:
                bandit_dict[my_last_action]['my_continue'] += 1
            else:
                bandit_dict[my_last_action]['my_continue'] = 0
            if op_action_list[-1] == op_action_list[-2]:
                bandit_dict[op_last_action]['op_continue'] += 1
            else:
                bandit_dict[op_last_action]['op_continue'] = 0
        
        if last_reward > 0:
            my_pull = my_last_action
        else:
            if observation['step'] >= 4:
                if (my_action_list[-1] == my_action_list[-2] == my_action_list[-3]):
                    if random.random() < 0.5:
                        my_pull = my_action_list[-1]
                    else:
                        my_pull = curr_agent()
                else:
                    my_pull = curr_agent()
            else:
                my_pull = curr_agent()
    
    return my_pull

In [ ]:
# Make a tar.gz file for submission
!tar cvfz main.py.tar.gz main.py model.sav

Thanks for reading out my notebook.. Hope it can help.. Please give suggestions also, Thanks in advance